In [1]:
# ------------Create Avec Matrix ---------------
# INPUT :
# - Ahat is estimated parameter of AR model. 
#   Ahat = (A1,A2,...,Ap)  has size (n,n,p)
# OUTPUT :
# - Avec is estimated A matrix into a vector form.
#   Avec has size n^2*p x 1 (ดู ได้จากโครงงานไอ้นัด theta)
import numpy as np

def ARLSvec(Ahat):
    # parameter 
    [p,n,n] = np.shape(Ahat) # Ahat = [A1 A2 ... Ap] that Ai is nxn matrix
    
    # convert Ahat matrix into a vector form
    Avec = []
    for i in range(n):
        for j in range(n):
            for k in range(p):
                Avec.append(Ahat[k][i,j]) # Avec is theta

    return Avec

In [2]:
# ------------create Avar Matrix ---------------
# INPUT:
# - data is the datasets of EEG signal, data = [y(1) y(2) ... y(N)]. 
#   yx(t) is the value of data at time equals to t.
# example: 
#             | y1(1) y1(2) .... Y1(N) |
#             | y2(1) y2(2) .... y2(N) |    
#      data = |   .     .          .   |  ; n is the dimension of time series, 
#             |   .     .          .   |    N is the number of time points in the data
#             | yn(1) yn(2) .... yn(N) |
# 
# - p is the lag-order of AR model.
# - Sigma is covariance matrix.

# OUTPUT:
# - Avar is consistent estimate of the asymptotic covariance matrix of Avec.
#   Avar has size n^2*p x n^2*p (ดู ได้จากโครงงานไอ้นัด แถวๆ Hbar)
import numpy as np

def ARLSvar(data,p,Sigma):
    #  parameter
    [n,N] = np.shape(data)

    # compute Avar from   
    ivAvar = zeros(n*n*p)
    for t in range(p,N):
        Yit = data[:,t-p:t] # [ y(t-p) ... y(t-2) y(t-1) ] 
        Yflip = fliplr(Yit) # [ y(t-1) y(t-2) ... y(t-p) ] 
        
        ybar = Yflip.reshape(-1,1) # 1 column automatically calculate the appropriate number of rows.
        Hbar = kron(identity(n),ybar.conj().transpose())

        ivSigma = np.linalg.inv(Sigma)
        ivAvar = ivAvar + np.dot(np.dot(Hbar.conj().transpose(),ivSigma),Hbar)

    Avar = np.linalg.inv(ivAvar)
    return Avar

In [3]:
# ------------Inverse-chi-squared distribution function---------------
# INPUT :
# - x is 1-alpha
# - p is degrees of freedom.

# OUTPUT :
# - c is a critical value
from scipy.stats import gamma

def chi2inv(x, p):
    inv = gamma.ppf(x,p/2,scale=2)    
    return inv    

In [4]:
# ----------- Wald test for AR model with--------------
# INPUT :
# - n is the dimension of the model output
# - p is the lag-order of the model
# - Avec is estimated A matrix into a vector form. Avec has size n^2*p x 1.
# - Avar is a consistent estimate of the asymptotic covariance matrix of Aest
# - alpha is a significance level of the test

# OUTPUT :
# - S is zero structure of estimated model parameter
# - W is Wald
import numpy as np
from PIL import Image

def funcWaldTest(n,p,Avec,Avar,alpha):
    # --- Wald test ---
    W = zeros((n,n))
    c = chi2inv(1-alpha,p) # a critical value
    print("Critical value :",c) 

    Cvec = [c+1] * n
    Cdiag = np.diag(Cvec)

    W = W + Cdiag

    # create Bhat = [Bij] = r(thetahat) from Avec = (B11,B12,...,B1n);...;(Bn1,Bn2,...,Bnn)    
    # r = [rij] >> rij = r[i,j]
    r = {}
    ni = 0
    for i in range(n):
        for j in range(n):
            r[i,j] = Avec[ni*p:(ni+1)*p] 
            ni+=1
            
    # create Wald matrix        
    nsquar = 0
    for i in range(n): #(1:n)
        for j in range(n): #(1:n)
            rij = np.array(r[i,j])            
            Avarij = Avar[nsquar*p:(nsquar+1)*p, nsquar*p:(nsquar+1)*p]
            invAvarij = np.linalg.inv(Avarij)
            W[i,j] = np.dot(np.dot(rij.conj().transpose(),invAvarij),rij) # Wald statistic value
            nsquar = nsquar+1

    S = W > c # True, False structure
    S = S*1   # 0, 1 structure

    # diagonal entries of S should be assigned to 1
    for i in range(n):
        S[i,i]=1

    return [S,W]